## 쇼핑몰

In [39]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [40]:
df_pur = pd.read_csv("ds_purchase_log.csv")
df_cust = pd.read_csv("ds_customer_mst.csv")
df_prod = pd.read_csv("ds_product_mst.csv")

In [41]:
df_cust_max = df_cust[["cust_no","age_gender","update_dt"]].groupby("cust_no").max()
base = pd.merge(df_pur,df_cust_max, how="left", on="cust_no")
base = pd.merge(base,df_prod, how="left", on="item_no")
base.head()
base.shape

(27910, 7)

#### [문제 1] (base_table을 활용하여 분석을 수행하시오.) 성별에 따라 어떤 대카테고리에 더 많은 매출이 발생하는지 비교하고자 한다. 매출의 비교는 대카테고리별 인당 매출금액 기준으로하며, 인당 매출금액은 매출 총 금액을 매출을 발생시킨 회원수를 나눈 값으로 한다. 남녀 간 비교를 위해 회원 연령대, 성별(age_gender)을 활용하여 성별 컬럼을 만들고, 성별 대 카테고리별 인당 구매금액을 산출하여, 남성 1인당 매출액 2위인 카테고리(A)와, 여성 1인당 매출액 2위인 카테고리(B)를 차례로 쓰시오. (10점)

In [42]:
base["sex"] = base["age_gender"].apply(lambda x : str(x)[-1:])
base.head()

,cust_no,item_no,age_gender,update_dt,item_ctg1,item_ctg2,item_amt,sex
0,1,1322,40대 M,20180829,의류,남성의류,94900.0,M
1,2,2273,40대 M,20180829,의류,남성의류,99900.0,M
2,2,2275,40대 M,20180829,의류,유아의류,36700.0,M
3,3,1309,40대 F,20180831,식품,간식,26000.0,F
4,3,605,40대 F,20180831,의류,남성의류,98600.0,F


In [43]:
def get_cnt(ctgr,sex):
    return len(base.loc[(base.item_ctg1 == ctgr) & (base.sex == sex)].cust_no.value_counts())

In [44]:
df_m = base.loc[base["sex"] == 'M', ["item_ctg1", "item_amt"]].groupby("item_ctg1").agg(sum)
df_m = df_m.reset_index()
df_m["cust_no"] = df_m["item_ctg1"].apply(lambda x : get_cnt(x,'M'))
df_m["total_sales"] = df_m["item_amt"]/df_m["cust_no"]
df_m = df_m.sort_values("total_sales",ascending=False).reset_index(drop=True)
df_m.head(2)

,item_ctg1,item_amt,cust_no,total_sales
0,가전,130788200.0,255,512894.901961
1,의류,604807600.0,7315,82680.464798


In [45]:
df_f = base.loc[base["sex"]=='F', ["item_ctg1","item_amt"]].groupby("item_ctg1").agg(sum)
df_f = df_f.reset_index()
df_f["cust_no"] = df_f["item_ctg1"].apply(lambda x : get_cnt(x,'F'))
df_f["total_sales"] = df_f["item_amt"]/df_f["cust_no"]
df_f = df_f.sort_values("total_sales",ascending=False).reset_index(drop=True)
df_f.head(2)

,item_ctg1,item_amt,cust_no,total_sales
0,가전,119879200.0,231,518957.575758
1,화장품,20847200.0,245,85090.612245


A:의류 B: 화장품

#### [문제 2] (base_table 또는 ds_purchase_log.csv을 활용하여 분석을 수행하시오.) 상위 20퍼센트의회원이 일으킨 매출은 전체 매출의 몇 퍼센트를 차지하며, 상위 20퍼센트의회원이 일으킨 매출은 하위 20퍼센트의 회원이 일으킨 매출의 몇 배인지 분석하고자 한다. 상위 20퍼센트와 하위 20퍼센트를산정하는 방식은 다음과 같다.

In [46]:
# (a) 전체 회원에 대해 회원별 구매금액에 대해 내림차순으로 순위를 매긴다. 순위는 동차발생시 동차로 등수를 주고 
# 그 다음 순위는 동차 사람수만큼 더한 이후 그 다음 순위를준다 (아래 예시 참조)
#  <순위예시>
#  금액     순위
#  5300     1
#  5300     1
#  5300     1
#  5200     4
#  5200     4

In [47]:
df_sales = base[["cust_no","item_amt"]].groupby("cust_no").agg(sum)
df_sales
df_sales["rank"] = df_sales["item_amt"].rank(ascending = False, method = "min")
df_sales = df_sales.sort_values("rank").reset_index()
df_sales.head()

,cust_no,item_amt,rank
0,6932,2394000.0,1.0
1,4401,1978900.0,2.0
2,2968,1930700.0,3.0
3,2223,1795200.0,4.0
4,12049,1716000.0,5.0


In [11]:
# (b) 상위 20퍼센트는 Rank <= N*0.2 인 회원을 의미하며, 하위 20퍼센트는 Rank >= N*0.8 인 회원을 의미한다. 
# (여기서 N은 base_table또는 ds_purchase_log.csv에 구매이력이 존재하는 회원수, Rank는 위 (a)에서 만든 순위를 의미한다)
# 상위 20퍼센트의 회원이 일으킨 매출은 전체 매출의 몇 퍼센트(A)이며, 상위 20퍼센트의회원이 일으킨 매출은 하위 20퍼센트의 
# 회원이 일으킨 매출의 몇 배(B)인지 차례대로 쓰시오. (10점)

# - Hint : Brightics는 rank() over(order by) 활용
# - A, B는 소수점 아래는 버리고,정수형으로 기술하시오.

In [48]:
N = df_sales["rank"].max()

len(df_sales.loc[df_sales["rank"] <= N*0.2])
len(df_sales.loc[df_sales["rank"] >= N*0.8])

df_sales.loc[df_sales["rank"] <= N*0.2].item_amt.agg(sum)
df_sales.loc[df_sales["rank"] >= N*0.8].item_amt.agg(sum)

90639900.0

In [13]:
(df_sales.loc[df_sales["rank"] <= N*0.2,"item_amt"].sum() / df_sales["item_amt"].sum() *100).astype('int')

49

In [14]:
(df_sales.loc[df_sales["rank"] <= N*0.2,"item_amt"].sum() / df_sales.loc[df_sales["rank"] >= N*0.8,"item_amt"].sum()).astype('int')

9

#### [문제 3] (base_table을 활용하여 분석을 수행하시오.) 회원별 구매금액이 연령대별로 차이가 있는지 비교하고자 회원 연령대, 성별(age_gender)을 활용하여 연령대(age_cls) 컬럼을 만들고 다음 분석을 수행한다.(15점)
 - 각 회원별 구매한 총 구매금액(y)을 산출하고, 회원의 연령대 그룹을 매핑한다. 
 - 연령대 그룹(x)은 age_cls가 '30대'이면 그룹이 ‘G1’, age_cls가 '40대'이면 그룹이 ‘G2’, age_cls가 그 외 연령대이면 그룹이 ‘G3’이다. 
 - 연령대 그룹(x)에 따라 회원별 구매한 총 구매금액(y)이 차이가 있는지 ANOVA를 수행하고, ANOVA의 F-통계량 추정치(A)를 구하시오. 
 - 사후검정으로 Duncan Test(alpha=0.05)를 수행하여 평균구매금액 순으로 오름차순하고, 그 결과를 등호, 부등호로 표기하시오. 

In [15]:
def def_age(s):
    if s == "30":
        return 'G1'
    if s == "40": 
        return 'G2'    
    else:
        return 'G3'        

In [16]:
base["age_cls"] = base["age_gender"].apply(lambda x : def_age(x[:2]))
base

df_total = base[["cust_no","age_cls","item_amt"]].groupby(["cust_no","age_cls"]).agg(sum)
df_total = df_total.reset_index()

df_anv = df_total[["age_cls","item_amt"]] 
df_anv = df_anv.rename(columns={"age_cls":"x",
                       "item_amt":"y"})
df_anv                        

# 연령대 그룹(x)에 따라 회원별 구매한 총 구매금액(y)이 차이가 있는지 ANOVA를 수행하고, 
# ANOVA의 F-통계량 추정치(A)를 구하시오. A는 소수점 아래는 버리고, 정수형으로 기술하시오.
from scipy.stats import f_oneway

stats, p_value = f_oneway(df_anv.loc[df_anv["x"]=="G1","y"],
                          df_anv.loc[df_anv["x"]=="G2","y"],
                          df_anv.loc[df_anv["x"]=="G3","y"])

stats.astype('int')

11

In [17]:
# 또한, 사후검정으로
# Brightics: Tukey's Range Test(alpha=0.05)
# R: Duncan Test(alpha=0.05)를
# 를 수행하여 평균구매금액 순으로 오름차순하고, 그 결과를 등호, 부등호로 표기하시오(B). (15점)
# - B는 회원별 구매한 총 구매금액(y)의평균이 그룹 G1, G2, G3 순으로 크고, 
# Test 결과가 1, 2가 같다면 G1=G2>G3으로 표기한다.

from statsmodels.stats import multicomp

result = multicomp.pairwise_tukeyhsd(df_anv["y"], df_anv["x"], alpha=0.05)
result.summary()

group1,group2,meandiff,lower,upper,reject
G1,G2,8461.1735,4117.8207,12804.5262,True
G1,G3,717.8831,-3454.2396,4890.0058,False
G2,G3,-7743.2904,-12467.8902,-3018.6906,True


#### [문제 4] (base_table을 활용하여 분석을 수행하시오.) 함께 구매되는 상품이 어떤지 분석하여 상품 상세 페이지에 ‘함께 구매하는 상품’으로 추천하기 위해, 구매 로그의 중카테고리 기준으로 연관성 분석(Association Rule)을 수행하고자 한다. 하나의 중카테고리만 구매한 이력이 있는 회원은 제외하고, 둘 이상의 중카테고리를 구매한 회원의 구매 이력을 기반으로 연관성 분석을 수행하고, 중카테고리가 ‘농산물’인 상품의 상세 페이지에 함께 노출되면 좋을 Lift 기준의 가장 적합한 중카테고리를 구하시오. (단, Min Support는 0.001 이상, Min Confidence는 0.001 이상 이어야 한다.) (15점)

In [18]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [19]:
base.head()

,cust_no,item_no,age_gender,update_dt,item_ctg1,item_ctg2,item_amt,sex,age_cls
0,1,1322,40대 M,20180829,의류,남성의류,94900.0,M,G2
1,2,2273,40대 M,20180829,의류,남성의류,99900.0,M,G2
2,2,2275,40대 M,20180829,의류,유아의류,36700.0,M,G2
3,3,1309,40대 F,20180831,식품,간식,26000.0,F,G2
4,3,605,40대 F,20180831,의류,남성의류,98600.0,F,G2


In [20]:
df_item = base[["cust_no", "item_ctg2"]].drop_duplicates()
df_sample = df_item.head()
df_sample

,cust_no,item_ctg2
0,1,남성의류
1,2,남성의류
2,2,유아의류
3,3,간식
4,3,남성의류


In [21]:
df_sample.groupby(['cust_no']).agg(lambda x: list(x))

,item_ctg2
cust_no,
1,[남성의류]
2,"[남성의류, 유아의류]"
3,"[간식, 남성의류]"


In [22]:
df_item = df_item.groupby(['cust_no']).agg(lambda x: list(x))

In [23]:
df_item.head(6)

,item_ctg2
cust_no,
1,[남성의류]
2,"[남성의류, 유아의류]"
3,"[간식, 남성의류]"
4,[유아의류]
5,"[화장지, 남성의류]"
6,[기타의류]


In [24]:
te = TransactionEncoder()
te_ary = te.fit(df_item['item_ctg2']).transform(df_item['item_ctg2'])

In [25]:
df_rule = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df_rule, min_support = 0.001, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.009076,(가공식품)
1,0.008656,(간식)
2,0.018572,(건강식품)
3,0.055926,(과자류)
4,0.045066,(기타의류)


In [26]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold = 0.001)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(농산물),(가공식품),0.022664,0.009076,0.001574,0.069444,7.651333,0.001368,1.064873
1,(가공식품),(농산물),0.009076,0.022664,0.001574,0.173410,7.651333,0.001368,1.182371
2,(남성의류),(건강식품),0.204921,0.018572,0.001259,0.006144,0.330843,-0.002547,0.987496
3,(건강식품),(남성의류),0.018572,0.204921,0.001259,0.067797,0.330843,-0.002547,0.852903
4,(농산물),(건강식품),0.022664,0.018572,0.001102,0.048611,2.617447,0.000681,1.031574


In [27]:
df_farm = rules.loc[rules['antecedents'] == rules.iloc[1,0], ]
df_farm.sort_values(by = ['lift'], ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(가공식품),(농산물),0.009076,0.022664,0.001574,0.17341,7.651333,0.001368,1.182371


------------------

#### [정답]
1 : 의류,화장품 <br>
2 : 49,9 <br>
3 : 11,G2>G3=G1 <br>
4 : 가공식품 <br>